In [1]:
import pandas as pd
import numpy as np
import os
import folium
from folium import plugins
import webbrowser
import chardet
from datetime import datetime

# rawdata = ""
# with open('2021_MCM_Problem_C_Data/2021MCMProblemC_DataSet.xlsx', 'r') as f:
#     rawdata += f.readline()

# chardet.detect(rawdata)

data = pd.read_excel("2021_MCM_Problem_C_Data/2021MCMProblemC_DataSet.xlsx", datetime = "Detection Date")

for i in range(0, len(data)) :
    if(isinstance(data.iloc[i]["Detection Date"], datetime)) : 
        data.loc[i, "Detection Date"] = data.iloc[i]["Detection Date"].year
    else :
        data.loc[i, "Detection Date"] = np.nan

PosData2020 = data[(data["Detection Date"] == 2020) & (data["Lab Status"] == "Positive ID")]

PosData2019 = data[(data["Detection Date"] == 2019) & (data["Lab Status"] == "Positive ID")]

In [2]:
print("2019: {0} : {1}".format(PosData2019["Latitude"].mean(), PosData2019["Longitude"].mean()))

print("2020: {0} : {1}".format(PosData2020["Latitude"].mean(), PosData2020["Longitude"].mean()))

2019: 49.024412000000005 : -122.96909459999999
2020: 48.959598666666665 : -122.59492977777775


In [6]:
import matplotlib.pyplot as plt
from bokeh.plotting import figure, save
from bokeh.models import GeoJSONDataSource
import webbrowser
import json


with open("us-all-mainland.geo.json", encoding="utf8") as f:
    temp_data = json.loads(f.read())

def wgs84_to_web_mercator(df, lon="Longitude", lat="Latitude"):

    k = 6378137
    df.loc[:, "x"] = df[lon] * (k * np.pi/180.0)
    df.loc[:, "y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k

    return df

PosData2020.loc[:, "x"] = PosData2020["Longitude"]

PosData2020.loc[:, "y"] = PosData2020["Latitude"]

PosData2019.loc[:, "x"] = PosData2019["Longitude"]

PosData2019.loc[:, "y"] = PosData2019["Latitude"]

def isInWA(district):
    if 'woe-name' in district['properties'] and 'Washington' in district['properties']['woe-name']:
        return True
    
    return False

temp_data['features'] = list(filter(isInWA, temp_data['features']))

Interval_Lat = 111.11

Interval_Long = 111.11 * np.cos((50 + 45) / 2 / 180 * 3.1415926)

L = 25

WA_Latitude_range = np.arange(45, 50, L / Interval_Lat)

WA_Longitude_range = -np.arange(116, 125, L / Interval_Long)

dp_array = np.zeros(shape =  (len(WA_Latitude_range), len(WA_Longitude_range)))

WA_geo_source = GeoJSONDataSource(geojson=json.dumps(temp_data))

p = figure(width=500, height=500)

p.scatter(PosData2020["x"], PosData2020["y"], color = "blue")

p.scatter(PosData2019["x"], PosData2019["y"], color = "red")

for x in WA_Longitude_range :
    p.line([x, x], [WA_Latitude_range[0], WA_Latitude_range[-1]])

for y in WA_Latitude_range :
    p.line([WA_Longitude_range[0], WA_Longitude_range[-1]], [y, y], color = "black")

p.patches(xs='xs', ys='ys', source=WA_geo_source)

save(obj=p, filename="WA_Map.html")

webbrowser.open("WA_Map.html")

True